# Creating Structured Meshes in Pointwise For Single Bifurcations

In [1]:
import pymethods as pma
import pymethods.pyplot as plt
import numpy as np
import matplotlib
from pymethods.algorithms.transfinite_interpolation import Transfinite2d
%matplotlib widget
# Load surface points generated from our statistical shape model
shape = np.load("shape.npy")/1000

To create our structured domain we need to create multiple quadrants for our mesh using the contours of our artery surface

In [2]:
# let's create a function which creates a square within some contour
def get_square_contour(contour, corner=[0.25, 0.25, 0], size=0.5):
    contour                  = pma.arrays.FlatContour(contour)
    transfinite_interpolator = Transfinite2d(contour)
    square                   = pma.arrays.contourShapes.Square(size, npts=100, corner=corner)
    square_edges             = square.transfinitable_corner_split()
    contour_edges            = [transfinite_interpolator(*edge[0:2]) for edge in square_edges]
    return contour_edges
def get_inner_outer_connector(outer_edges, inner_edges):
    output = []
    for i, (outer, inner) in enumerate(zip(outer_edges, inner_edges)):
        if i < 3:
            output.append(pma.arrays.Vectorspace(np.stack((outer[:, 0], inner[:, 0]), axis=-1)))
        else:
            output.append(pma.arrays.Vectorspace(np.stack((outer_edges[1][:, -1], inner_edges[1][:, -1]), axis=-1)))
    return output

Bellow we show the OH topology created via transfinite interpolation

In [12]:
# plot the inlet contour and the interpolated edges
plt.close()
inlet_contour  = pma.arrays.Contour(shape[:, :, 0])
outlet_contour = pma.arrays.Contour(shape[:, :, -1])

inner_inlet_edges  = get_square_contour(shape[:, :, 0])
inner_outlet_edges = get_square_contour(shape[:, :, -1])

# cannot use corners of 0 or size of 1 as of yet
outer_inlet_edges   = get_square_contour(shape[:, :, 0], corner=[0.000001, 0.000001, 0], size=0.99999)
outer_outlet_edges  = get_square_contour(shape[:, :, -1], corner=[0.000001, 0.000001, 0], size=0.99999)

# create the inner to outer connectors
inlet_inner_outer_connector_edges  = get_inner_outer_connector(outer_inlet_edges, inner_inlet_edges)
outlet_inner_outer_connector_edges = get_inner_outer_connector(outer_outlet_edges, inner_outlet_edges)

fig = plt.figure(figsize=plt.figaspect(0.175))
axes = [fig.add_subplot(1,3,i, projection="3d") for i in range(1, 3)]

[edge.plot3d(ax=axes[0], c="red") for edge in inner_inlet_edges]
[edge.plot3d(ax=axes[0], c="blue") for edge in outer_inlet_edges]
[edge.plot3d(ax=axes[0], c="green") for edge in inlet_inner_outer_connector_edges]
[edge[:, 0, None].plot3d(".", ax=axes[0], c="blue") for edge in inlet_inner_outer_connector_edges]

[edge.plot3d(ax=axes[1], c="red") for edge in inner_outlet_edges]
[edge.plot3d(ax=axes[1], c="blue") for edge in outer_outlet_edges]
[edge.plot3d(ax=axes[1], c="green") for edge in outlet_inner_outer_connector_edges]
[edge[:, 0, None].plot3d(".", ax=axes[1], c="blue") for edge in outlet_inner_outer_connector_edges]

plt.show()

5.921354280977886e-06


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Let's now create the edges for the innner and outer domains of our oh topology

In [4]:
# We will now extract the connectors which make the boundaries of the inner and outer domains
inner_block_edges = []
outer_block_edges = []
for i in range(shape.shape[-1]):
    contour = pma.arrays.FlatContour(
        shape[:,:, i]
    ).filter(11, 3)

    transfinite_interpolator = Transfinite2d(contour)
    square = pma.arrays.contourShapes.Square(
        0.5, npts=100, corner=[0.25, 0.25, 0])[0:2]

    npts = 100

    middle = transfinite_interpolator(*square)

    connector1 = transfinite_interpolator(
        np.linspace(0, 0.25, npts), np.linspace(0, 0.25, npts))
    connector2 = transfinite_interpolator(
        np.linspace(0, 0.25, npts), np.linspace(1, 0.75, npts))
    connector3 = transfinite_interpolator(
        np.linspace(1, 0.75, npts), np.linspace(0, 0.25, npts))
    connector4 = transfinite_interpolator(
        np.linspace(1, 0.75, npts), np.linspace(1, 0.75, npts))

    inner_block_edges.append(
        np.stack((connector1[:, -1],connector2[:, -1],connector3[:, -1],connector4[:, -1]), axis=0)
    )
    outer_block_edges.append(
        np.stack((connector1[:, 0],connector2[:, 0],connector3[:, 0],connector4[:, 0]), axis=0)
    )
outer_block_edges = np.stack(outer_block_edges, axis=-1)
inner_block_edges = np.stack(inner_block_edges, axis=-1)


In [5]:
# we need to make sure thate the connectors are flush against eash other
def get_closest_point(point, comparison):
    distances = np.linalg.norm(comparison - point.squeeze()[:, None], axis=0)
    return np.argmin(distances)

inner_inlet_points  = np.concatenate(inner_inlet_edges, axis=-1)
inner_outlet_points = np.concatenate(inner_outlet_edges, axis=-1)

outer_inlet_points  = np.concatenate(outer_inlet_edges, axis=-1)
outer_outlet_points = np.concatenate(outer_outlet_edges, axis=-1)

for i in range(len(inner_block_edges)):
    inner_block_edges[i][:, 0]  = inner_inlet_points[:, get_closest_point(inner_block_edges[i][:, 0], inner_inlet_points)]
    inner_block_edges[i][:, -1] = inner_outlet_points[:, get_closest_point(inner_block_edges[i][:, -1], inner_outlet_points)]

for i in range(len(outer_block_edges)):
    outer_block_edges[i][:, 0]  = outer_inlet_points[:, get_closest_point(outer_block_edges[i][:, 0], outer_inlet_points)]
    outer_block_edges[i][:, -1] = outer_outlet_points[:, get_closest_point(outer_block_edges[i][:, -1], outer_outlet_points)]

inner_block_edges = [pma.arrays.Vectorspace(edge) for edge in inner_block_edges]
outer_block_edges = [pma.arrays.Vectorspace(edge) for edge in outer_block_edges]

Bellow is a utility function for plotting the boundary edges

In [6]:
def plot_outer_blocks(ids, ax, color, **kwargs):
    inner_inlet_edges[ids[0]].plot3d(ax=ax, c=color, **kwargs)
    outer_inlet_edges[ids[0]].plot3d(ax=ax, c=color, **kwargs)
    inner_outlet_edges[ids[0]].plot3d(ax=ax, c=color, **kwargs)
    outer_outlet_edges[ids[0]].plot3d(ax=ax, c=color, **kwargs)

    inlet_inner_outer_connector_edges[ids[1][0]].plot3d(ax=ax, c=color, **kwargs)
    inlet_inner_outer_connector_edges[ids[1][1]].plot3d(ax=ax, c=color, **kwargs)
    outlet_inner_outer_connector_edges[ids[1][0]].plot3d(ax=ax, c=color, **kwargs)
    outlet_inner_outer_connector_edges[ids[1][1]].plot3d(ax=ax, c=color, **kwargs)

    inner_block_edges[ids[2][0]].plot3d(ax=ax, c=color, **kwargs)
    inner_block_edges[ids[2][1]].plot3d(ax=ax, c=color, **kwargs)
    outer_block_edges[ids[2][0]].plot3d(ax=ax, c=color, **kwargs)
    outer_block_edges[ids[2][1]].plot3d(ax=ax, c=color, **kwargs)

Now we can plot the scaffolding we have generated for our domain

In [7]:
f, ax = plt.figure_3d()
a = 0.5
plot_outer_blocks([0, (0, 1), (0, 2)], ax, "red", alpha=a)
plot_outer_blocks([1, (1, 3), (2, 3)], ax, "blue", alpha=a)
plot_outer_blocks([2, (2, 3), (1, 3)], ax, "green", alpha=a)
plot_outer_blocks([3, (0, 2), (0, 1)], ax, "purple", alpha=a)

plt.equal_aspect_3d()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Finally we can create the meshes programatically in pointwise

In [8]:
# a utility tool for creating connectors with pointwise
def create_connectors(api, connector_list, dims):
    connectors = []
    with api.Application.begin("Create") as creator:
        for edge in connector_list:
            seg = api.SegmentSpline()
            [seg.addPoint(point) for point in edge.T]
            con = api.Connector()
            con.addSegment(seg)
            con.setDimension(dims)
            connectors.append(con)
    creator.end()
    return connectors

In [11]:
from pointwise import GlyphClient
# glf = GlyphClient(port=2807)
glf = GlyphClient(port=0)
# pointwise api
pw = glf.get_glyphapi()
# reset the client
_ = pw.Application.reset()

# create the inlet and outlet edges for the inner region
inner_inlet_connectors = create_connectors(pw, inner_inlet_edges, 15)
inner_outlet_connectors = create_connectors(pw, inner_outlet_edges, 15)
# create the boundary of the inner region
inner_block_connectors = create_connectors(pw, inner_block_edges, 150)

# create the inlet and outlet edges for the outer region
outer_inlet_connectors = create_connectors(pw, outer_inlet_edges, 15)
outer_outlet_connectors = create_connectors(pw, outer_outlet_edges, 15)
# create the boundary of the outer region
outer_block_connectors = create_connectors(pw, outer_block_edges, 150)
    
# create the connectors that connect the inner and outer regions of the domain
inlet_inner_outer_connectors = create_connectors(pw, inlet_inner_outer_connector_edges, 15)
outlet_inner_outer_connectors = create_connectors(pw, outlet_inner_outer_connector_edges, 15)

def create_outer_blocks(ids):
    domains = []
    # keep track of the domains on the inlet, outlet and walls to make setting the bc easier
    inlet_patch = pw.DomainStructured.createFromConnectors(
            [
                inner_inlet_connectors[ids[0]],
                outer_inlet_connectors[ids[0]],
                inlet_inner_outer_connectors[ids[1][0]],
                inlet_inner_outer_connectors[ids[1][1]]
            ]
        )

    outlet_patch = pw.DomainStructured.createFromConnectors(
            [
                inner_outlet_connectors[ids[0]],
                outer_outlet_connectors[ids[0]],
                outlet_inner_outer_connectors[ids[1][0]],
                outlet_inner_outer_connectors[ids[1][1]]
            ]
        )
    wall_patch = pw.DomainStructured.createFromConnectors(
            [
                outer_inlet_connectors[ids[0]],
                outer_outlet_connectors[ids[0]],
                outer_block_connectors[ids[2][0]],
                outer_block_connectors[ids[2][1]],
            ]
    )
    domains.extend(
        pw.DomainStructured.createFromConnectors(
            [
                inlet_inner_outer_connectors[ids[1][0]],
                outlet_inner_outer_connectors[ids[1][0]],
                inner_block_connectors[ids[2][0]],
                outer_block_connectors[ids[2][0]]
            ]
        )
    )
    domains.extend(
        pw.DomainStructured.createFromConnectors(
            [
                inlet_inner_outer_connectors[ids[1][1]],
                outlet_inner_outer_connectors[ids[1][1]],
                inner_block_connectors[ids[2][1]],
                outer_block_connectors[ids[2][1]]
            ]
        )
    )

    domains.extend(
        pw.DomainStructured.createFromConnectors(
            [
                inner_inlet_connectors[ids[0]],
                inner_outlet_connectors[ids[0]],
                inner_block_connectors[ids[2][0]],
                inner_block_connectors[ids[2][1]],
            ]
        )
    )

    domains.extend(inlet_patch)
    domains.extend(outlet_patch)
    domains.extend(wall_patch)

    return domains, inlet_patch, outlet_patch, wall_patch

all_domains = []
inlet_domains = []
outlet_domains = []
wall_domains = []
# create the inner inlet and outlet domains
inner_inlet_domain  = pw.DomainStructured.createFromConnectors(inner_inlet_connectors)
inner_outlet_domain = pw.DomainStructured.createFromConnectors(inner_outlet_connectors)
inlet_domains.extend(inner_inlet_domain)
outlet_domains.extend(inner_outlet_domain)
all_domains.extend(inner_inlet_domain)  
all_domains.extend(inner_outlet_domain)
# create the domains for the outer blocks using the connectors, which which we know the order of already
connector_ids = [
    [0, (0, 1), (0, 2)], [1, (1, 3), (2, 3)], [2, (2, 3), (1, 3)], [3, (0, 2), (0, 1)]
]
for ids in connector_ids:
    all_doms, inlet, outlet, wall = create_outer_blocks(ids)
    inlet_domains.extend(inlet)
    outlet_domains.extend(outlet)
    wall_domains.extend(wall)
    all_domains.extend(all_doms)

# now create the blocks
blocks = pw.BlockStructured.createFromDomains(all_domains)

# create the openfoam case
pw.Application.setCAESolver("OpenFOAM")

wall_bc = pw.BoundaryCondition.create()
wall_bc.setName("WALL")
wall_bc.setPhysicalType("-usage", "CAE", "wall")

inlet_bc = pw.BoundaryCondition.create()
inlet_bc.setName("INLET")
inlet_bc.setPhysicalType("-usage", "CAE", "patch")

outlet_bc = pw.BoundaryCondition.create()
outlet_bc.setName("OUTLET")
outlet_bc.setPhysicalType("-usage", "CAE", "patch")

inlet_bc.apply([
    (pw.Block.getBlocksFromDomains(domain)[0], domain) for domain in inlet_domains
])

outlet_bc.apply([
    (pw.Block.getBlocksFromDomains(domain)[0], domain) for domain in outlet_domains
])

wall_bc.apply([
    (pw.Block.getBlocksFromDomains(domain)[0], domain) for domain in wall_domains
])

# now save the mesh
import pathlib as pt
save_folder = pt.Path("./test_foam/constant/polyMesh")
with pw.Application.begin("CaeExport") as cae_exporter:
    cae_exporter.addAllEntities()
    cae_exporter.initialize(
        "-strict", "-type", "CAE", save_folder.absolute().as_posix()
    )
    cae_exporter.verify()
    cae_exporter.write()